# MLP-Mixer

## 0. Paper

### Info
* Title: MLP-Mixer _ An all-MLP Architecture for Vision
* Author: Ilya Tolstikhin
* Task: Image Classification
* Link: https://arxiv.org/abs/2105.01601


### Features
* Dataset: CIFAR-10

### Reference
* ##


## 1. Setting

In [1]:
import os
from glob import glob
from tqdm.auto import tqdm

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torchsummary import torchsummary

In [2]:
class CONFIG:
    model_dim = 128
    patch_size = 8
    num_layers = 8
    batch_size = 128
    epoch_size = 200
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    dataset = 'cifar-10'
    base_dir = '/content/drive/Shared drives/Yoon/Project/Doing/Deep Learning Paper Implementation'

## 2. Data

In [3]:
def create_dataloader(batch_size):
    train_transform = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2439, 0.2616)),
    ])

    test_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2439, 0.2616)),
    ])


    train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
    test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)
    train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [45000, 5000])

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    return train_loader, val_loader, test_loader

In [4]:
train_loader, val_loader, test_loader = create_dataloader(CONFIG.batch_size)
inputs, targets = next(iter(train_loader))
inputs.size(), targets.size()

Files already downloaded and verified
Files already downloaded and verified


(torch.Size([128, 3, 32, 32]), torch.Size([128]))

## 3. Model

In [5]:
class Embedding(nn.Module):
    def __init__(self, dim_size, patch_size):
        super(Embedding, self).__init__()
        # num_patches = (image_size // patch_size) ** 2
        self.conv = nn.Conv2d(3, dim_size, kernel_size=patch_size, stride=patch_size)

    def forward(self, x):
        x = self.conv(x) # (batch_size, dim_size, img_size/patch_size, img_size/patch_size)
        x = x.flatten(2) # (batch_size, dim_size, num_patches)
        x = x.transpose(1, 2) # (batch_size, num_patches, dim_size)
        return x

class MLP(nn.Module):
    def __init__(self, features):
        super(MLP, self).__init__()
        self.linear1 = nn.Linear(features, features)
        self.gelu = nn.GELU()
        self.linear2 = nn.Linear(features, features)
    
    def forward(self, x):
        x = self.linear1(x)
        x = self.gelu(x)
        x = self.linear2(x)
        return x

class MixerLayer(nn.Module):
    def __init__(self, features, num_tokens):
        super(MixerLayer, self).__init__()
        self.features = features
        self.num_tokens = num_tokens

        self.norm1 = nn.LayerNorm(features)
        self.mlp1 = MLP(num_tokens)
        self.norm2 = nn.LayerNorm(features)
        self.mlp2 = MLP(features)

    
    def forward(self, x):
        shortcut = x
        x = self.norm1(x)
        x = x.transpose(1, 2)
        x = self.mlp1(x)
        x = x.transpose(1, 2)
        x += shortcut

        shortcut = x
        x = self.norm2(x)
        x = self.mlp2(x)
        x += shortcut
        return x

In [6]:
class MLPMixer(nn.Module):
    def __init__(self, model_dim, num_layers, patch_size, image_size=32, num_class=10):
        super(MLPMixer, self).__init__()
        num_tokens = (image_size // patch_size) ** 2
        self.embedding = Embedding(model_dim, patch_size)
        self.mixers = nn.Sequential(*[MixerLayer(model_dim, num_tokens) for _ in range(num_layers)])
        self.classifier = nn.Linear(model_dim, num_class)
    
    def forward(self, x):
        x = self.embedding(x)
        x = self.mixers(x)
        x = x.mean(dim=1)
        x = self.classifier(x)
        return x

## 4. Experiment

In [7]:
class AverageMeter(object):
    def __init__(self, name):
        self.name = name
        self.reset()

    def reset(self):
        self.sum = 0
        self.count = 0
        self.avg = 0

    def update(self, val, n=1):
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = f'{self.name:10s} {self.avg:.3f}'
        return fmtstr


class ProgressMeter(object):
    def __init__(self, meters):
        self.meters = [AverageMeter(m) for m in meters]
    
    def reset(self):
        for m in self.meters:
            m.reset()
    
    def update(self, values, n=1):
        for m, v in zip(self.meters, values):
            m.update(v, n)
            self.__setattr__(m.name, m.avg)

    def log(self):
        msg = [str(meter) for meter in self.meters]
        msg = ' | '.join(msg)
        return msg


def accuracy(logits, targets):
    _, preds = logits.max(1)
    acc = (preds == targets).float().mean()
    return acc

In [8]:
class Trainer(object):
    def __init__(self, model, criterion, optimizer, scheduler):
        self.model = model.to(CONFIG.device)
        self.criterion = criterion.to(CONFIG.device)
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.best_epoch, self.best_score = 0, 0
        

    def train(self, train_loader, epoch):
        progress = ProgressMeter(["train_loss", "train_acc"])
        self.model.train()

        pbar = tqdm(train_loader)
        pbar.set_description(f'TRAIN {epoch:03d}')
        for idx, (inputs, targets) in enumerate(pbar):
            inputs, targets = inputs.to(CONFIG.device), targets.to(CONFIG.device)
            outputs = self.model(inputs)
            loss = self.criterion(outputs, targets)
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()

            loss = loss.item()
            acc = accuracy(outputs, targets).item()
            progress.update([loss, acc], n=inputs.size(0))
            pbar.set_postfix(log=progress.log())

        self.scheduler.step()

    
    def validate(self, valid_loader, epoch):
        progress = ProgressMeter(["valid_loss", "valid_acc"])
        self.model.eval()

        pbar = tqdm(valid_loader)
        pbar.set_description(f'VALID {epoch:03d}')
        with torch.no_grad():
            for idx, (inputs, targets) in enumerate(pbar):
                inputs, targets = inputs.to(CONFIG.device), targets.to(CONFIG.device)
                outputs = self.model(inputs)
                loss = self.criterion(outputs, targets)
                loss = loss.item()
                acc = accuracy(outputs, targets).item()
                progress.update([loss, acc], n=inputs.size(0))
                pbar.set_postfix(log=progress.log())

            if progress.valid_acc > self.best_score:
                self.best_epoch = epoch
                self.best_score = progress.valid_acc
                ckpt = {
                    'best_epoch': self.best_epoch,
                    'best_score': self.best_score,
                    'model_state_dict': self.model.state_dict()
                }
                torch.save(ckpt, 'ckpt.pt')

    
    def test(self, test_loader):
        progress = ProgressMeter(["test_loss", "test_acc"])
        ckpt = torch.load('ckpt.pt')
        self.model.load_state_dict(ckpt['model_state_dict'])
        self.model.eval()

        pbar = tqdm(test_loader)
        pbar.set_description(f'TEST')
        with torch.no_grad():
            for idx, (inputs, targets) in enumerate(pbar):
                inputs, targets = inputs.to(CONFIG.device), targets.to(CONFIG.device)
                outputs = self.model(inputs)
                loss = self.criterion(outputs, targets)
                loss = loss.item()
                acc = accuracy(outputs, targets).item()
                progress.update([loss, acc], n=inputs.size(0))
                pbar.set_postfix(log=progress.log())

In [9]:
train_loader, valid_loader, test_loader = create_dataloader(CONFIG.batch_size)
model = MLPMixer(CONFIG.model_dim, CONFIG.num_layers, CONFIG.patch_size)
criterion = nn.CrossEntropyLoss()
#optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=1e-4)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, CONFIG.epoch_size)

trainer = Trainer(model, criterion, optimizer, scheduler)

Files already downloaded and verified
Files already downloaded and verified


In [10]:
for ep in range(CONFIG.epoch_size):
    print('=' * 75)
    trainer.train(train_loader, ep)
    trainer.validate(valid_loader, ep)

In [11]:
trainer.test(test_loader)